In [ ]:
from langchain.vectorstores import qdrant
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
import getpass
import os
import qdrant_client
from qdrant_client.http import models
from qdrant_client import QdrantClient

### Initilizing the Qdrant Cloud

In [ ]:
# Initialize the QdrantClient with the Qdrant server URL and API key

qdrant_client = QdrantClient(
    url="<--Enter Your Qdrant URL-->", 
    api_key="Enter Your Qdrant Api key",
)

### Creating a Collection


In [ ]:

qdrant_client.create_collection(
    collection_name="<-<-Give a name of your collection-->",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)

In [ ]:
os.environ["OpenAI_API_Key"] ="<<--ENTER YOUR API KEY-->>"
embeddings = OpenAIEmbeddings()
vector_store = Qdrant(
    client=qdrant_client, collection_name="new-collection", 
    embeddings=embeddings,
)



### Split the texts into Chunks

In [ ]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator = "/n",
        chunk_size=1000, 
        chunk_overlap= 200,
        length_function=len
        )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
with open('story.txt') as f:
    raw_text = f.read()

texts=get_chunks(raw_text)
vector_store.add_texts(texts)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
qa = RetrievalQA.from_chain_type( llm=OpenAI(), chain_type = "stuff",
                                 retriever= vector_store.as_retriever()
                                )

In [ ]:
querry = "Who is Luna?"
response = qa.run(querry)
print(response)